# Lectura 04: Importar polars

## Importar Polars

Usualmente la librería de Polars se importa de la siguiente manera:

In [1]:
import polars as pl

# Ejecutar la primera sentencia de Polars

In [2]:
df = pl.DataFrame(
     {
         "numeros": [1, 2, 3],
         "colores": ["rojo", "azul", "verde"],
         "pais": ["Cuba", "Mexico", "Francia"]
     }
 )

# Mostrar el DataFrame

In [3]:
df

numeros,colores,pais
i64,str,str
1,"""rojo""","""Cuba"""
2,"""azul""","""Mexico"""
3,"""verde""","""Francia"""


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
file_path = ('/content/drive/MyDrive/polars/seccion01/chess_games.csv')

In [6]:
pl.scan_csv(file_path).head()

###Leer un archivo polars

In [7]:
pl.scan_csv(file_path).head(3).collect()

Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
str,str,str,str,str,str,i64,i64,f64,f64,str,str,str,str,str
""" Classical ""","""eisaaaa""","""HAMID449""","""1-0""","""2016.06.30""","""22:00:01""",1901,1896,11.0,-11.0,"""D10""","""Slav Defense""","""300+5""","""Time forfeit""","""1. d4 d5 2. c4 c6 3. e3 a6 4. …"
""" Blitz ""","""go4jas""","""Sergei1973""","""0-1""","""2016.06.30""","""22:00:01""",1641,1627,-11.0,12.0,"""C20""","""King's Pawn Opening: 2.b3""","""300+0""","""Normal""","""1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 …"
""" Blitz tournament ""","""Evangelistaizac""","""kafune""","""1-0""","""2016.06.30""","""22:00:02""",1647,1688,13.0,-13.0,"""B01""","""Scandinavian Defense: Mieses-K…","""180+0""","""Time forfeit""","""1. e4 d5 2. exd5 Qxd5 3. Nf3 B…"


## Rapidez

In [8]:
import pandas as pd

In [9]:
%%timeit -n1 -r1
pl.read_csv(file_path)

26.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n1 -r1
pd.read_csv(file_path, engine='pyarrow')

42.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## es mas rapido que pandas con optimizaciones

## Optimizaciones de consultas en forma automatica

In [13]:
%%timeit -n1 -r1
(
pl.scan_csv(file_path)
.group_by('Event')
.agg(
    min_elo_white=pl.col('WhiteElo').mean(),
    max_elo_black=pl.col('BlackElo').mean()
)
).collect()


22.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [30]:
%%timeit -n1 -r1

 (
     pd.read_csv(file_path, engine='pyarrow', usecols=['Event', 'WhiteElo', 'BlackElo'])
     .groupby('Event')
     .agg(
         {
               'WhiteElo':['mean'],
               'BlackElo':['mean']
         }
 )


SyntaxError: incomplete input (<unknown>, line 9)

In [31]:
df1 = pl.DataFrame(
     {
         "num": [1, 2, 3],
         "colores": ["rojo", "azul", "verde"],
         "pais": ["Cuba", "Mexico", "Francia"]
     }
 )

In [33]:
print(df1)

shape: (3, 3)
┌─────┬─────────┬─────────┐
│ num ┆ colores ┆ pais    │
│ --- ┆ ---     ┆ ---     │
│ i64 ┆ str     ┆ str     │
╞═════╪═════════╪═════════╡
│ 1   ┆ rojo    ┆ Cuba    │
│ 2   ┆ azul    ┆ Mexico  │
│ 3   ┆ verde   ┆ Francia │
└─────┴─────────┴─────────┘


In [35]:
df1.columns

['num', 'colores', 'pais']